In [1]:
!pip install pandas
!pip install request
!pip install numpy
!pip install matplotlib
!pip install seaborn
!pip install sklearn
!pip install scipy
!pip install numpy

ERROR: Could not find a version that satisfies the requirement request (from versions: none)
ERROR: No matching distribution found for request
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


import numpy as np
import os
from scipy.sparse import coo_matrix

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_data = pd.read_csv('udemy.csv')
train_data.columns

Index(['id', 'title', 'url', 'is_paid', 'num_subscribers', 'avg_rating',
       'avg_rating_recent', 'rating', 'num_reviews', 'is_wishlisted',
       'num_published_lectures', 'num_published_practice_tests', 'created',
       'published_time', 'discount_price__amount', 'discount_price__currency',
       'discount_price__price_string', 'price_detail__amount',
       'price_detail__currency', 'price_detail__price_string'],
      dtype='object')

In [4]:
train_data.head(2)

,id,title,url,is_paid,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,is_wishlisted,num_published_lectures,num_published_practice_tests,created,published_time,discount_price__amount,discount_price__currency,discount_price__price_string,price_detail__amount,price_detail__currency,price_detail__price_string
0,762616,The Complete SQL Bootcamp 2020: Go from Zero t...,/course/the-complete-sql-bootcamp/,True,295509,4.66019,4.67874,4.67874,78006,False,84,0,2016-02-14T22:57:48Z,2016-04-06T05:16:11Z,455.0,INR,₹455,8640.0,INR,"₹8,640"
1,937678,Tableau 2020 A-Z: Hands-On Tableau Training fo...,/course/tableau10/,True,209070,4.58956,4.60015,4.60015,54581,False,78,0,2016-08-22T12:10:18Z,2016-08-23T16:59:49Z,455.0,INR,₹455,8640.0,INR,"₹8,640"


In [5]:
train_data.shape

(13608, 20)

In [6]:
train_data.isnull().sum()

,0
id,0
title,0
url,0
is_paid,0
num_subscribers,0
avg_rating,0
avg_rating_recent,0
rating,0
num_reviews,0
is_wishlisted,0


In [7]:
train_data = train_data[['id', 'title', 'is_paid', 'num_subscribers', 'avg_rating', 'avg_rating_recent', 'rating', 'num_reviews', 'is_wishlisted']]
train_data.head()

,id,title,is_paid,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,is_wishlisted
0,762616,The Complete SQL Bootcamp 2020: Go from Zero t...,True,295509,4.66019,4.67874,4.67874,78006,False
1,937678,Tableau 2020 A-Z: Hands-On Tableau Training fo...,True,209070,4.58956,4.60015,4.60015,54581,False
2,1361790,PMP Exam Prep Seminar - PMBOK Guide 6,True,155282,4.59491,4.59326,4.59326,52653,False
3,648826,The Complete Financial Analyst Course 2020,True,245860,4.54407,4.53772,4.53772,46447,False
4,637930,An Entire MBA in 1 Course:Award Winning Busine...,True,374836,4.47080,4.47173,4.47173,41630,False


In [8]:
train_data.isnull().sum()

,0
id,0
title,0
is_paid,0
num_subscribers,0
avg_rating,0
avg_rating_recent,0
rating,0
num_reviews,0
is_wishlisted,0


In [9]:
train_data.duplicated().sum()

0

In [10]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

def clean_and_extract_tags(text):
  doc = nlp(text.lower())
  tags = [token.text for token in doc if token.text.isalnum() and token.text not in STOP_WORDS]
  return ', '.join(tags)

column_to_extract_tag_from = ['title']

for column in column_to_extract_tag_from:
  train_data['tags'] = train_data[column].apply(clean_and_extract_tags)



In [11]:
train_data

,id,title,is_paid,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,is_wishlisted,tags
0,762616,The Complete SQL Bootcamp 2020: Go from Zero t...,True,295509,4.66019,4.67874,4.67874,78006,False,"complete, sql, bootcamp, 2020, zero, hero"
1,937678,Tableau 2020 A-Z: Hands-On Tableau Training fo...,True,209070,4.58956,4.60015,4.60015,54581,False,"tableau, 2020, z, hands, tableau, training, da..."
2,1361790,PMP Exam Prep Seminar - PMBOK Guide 6,True,155282,4.59491,4.59326,4.59326,52653,False,"pmp, exam, prep, seminar, pmbok, guide, 6"
3,648826,The Complete Financial Analyst Course 2020,True,245860,4.54407,4.53772,4.53772,46447,False,"complete, financial, analyst, course, 2020"
4,637930,An Entire MBA in 1 Course:Award Winning Busine...,True,374836,4.47080,4.47173,4.47173,41630,False,"entire, mba, 1, course, award, winning, busine..."
...,...,...,...,...,...,...,...,...,...,...
13603,3005408,מושגים בסיסיים באופציות חלק ב,False,47,0.00000,0.00000,0.00000,0,False,"מושגים, בסיסיים, באופציות, חלק, ב"
13604,3467284,Programa de Integridade,False,19,0.00000,0.00000,0.00000,0,False,"programa, de, integridade"
13605,3368222,Goodwill and Method of Calculating Goodwill (A...,False,47,0.00000,0.00000,0.00000,0,False,"goodwill, method, calculating, goodwill, accou..."
13606,3211345,Poderoso Investidor,False,48,0.00000,0.00000,0.00000,0,False,"poderoso, investidor"


##Rating Based Reccomendation

In [12]:
average_ratings = train_data.groupby(['title', 'num_reviews', 'is_wishlisted', 'num_subscribers'])['rating'].mean().reset_index()
top_rated_items = average_ratings.sort_values(by='rating', ascending = False)
rating_based_reccomendation = top_rated_items.head(20)

In [13]:
rating_based_reccomendation

,title,num_reviews,is_wishlisted,num_subscribers,rating
8111,Mais Dinheiro,2,False,31,5.0
2665,Como hacer empresa en 10 sesiones prácticas,7,False,44,5.0
8861,New! Raise funds for your innovative business ...,28,False,2233,5.0
8890,Not for Profit QuickBooks Desktop vs QuickBook...,1,False,5198,5.0
8922,OS MELHORES SISTEMAS PARA DAY TRADE!,1,False,38,5.0
884,Amazon FBA The Ultimate Private Label Business...,5,False,26,5.0
8961,Online Business Plan,3,False,502,5.0
12141,The Freelancing Online Business Model - Practi...,8,False,27,5.0
8990,Online Speeches and Presentation Preparation S...,3,False,8,5.0
4328,Excel para Bancários (do básico ao avançado),1,False,4,5.0


##Past Video and Search Based Reccomendation

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(train_data['tags'])

In [15]:
train_data.head(2)

,id,title,is_paid,num_subscribers,avg_rating,avg_rating_recent,rating,num_reviews,is_wishlisted,tags
0,762616,The Complete SQL Bootcamp 2020: Go from Zero t...,True,295509,4.66019,4.67874,4.67874,78006,False,"complete, sql, bootcamp, 2020, zero, hero"
1,937678,Tableau 2020 A-Z: Hands-On Tableau Training fo...,True,209070,4.58956,4.60015,4.60015,54581,False,"tableau, 2020, z, hands, tableau, training, da..."


In [16]:
cosine_similarities_content = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_similarities_content

array([[1.        , 0.07273969, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07273969, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [27]:
item_name = "Learn SQL with ease"
item_index = train_data[train_data['title'] == item_name].index[0]

In [28]:
similar_items = list(enumerate(cosine_similarities_content[item_index]))

In [29]:
similar_items = sorted(similar_items, key=lambda x: x[1], reverse=True)
top_similar_items = similar_items[1:10]

In [30]:
recommended_item_indices = [x[0] for x in top_similar_items]
recommended_items = train_data.iloc[recommended_item_indices][['title', 'rating']]

In [31]:
recommended_items

,title,rating
2273,Accounting 101: Learn Basic Accounting Concept...,4.21766
11156,Start Investing Quickly With Ease,0.00000
2465,Microsoft SQL Server And T-SQL Course For Begi...,4.45244
1594,SQL Essentials: The Beginner's Guide to SQL La...,3.93713
7104,Accounting 101: Learn The Entire Accounting Cy...,4.24415
4763,SQL for Beginners with Microsoft SQL Server Da...,4.72768
4378,Business and Data Analysis with SQL,4.57578
8050,Business Communication: Lead Effective Meeting...,4.14193
7947,Learning Path: SQL: Up and Running with SQL Se...,3.84633


In [32]:
train_data.to_csv('data.csv', index=False)

##Collaborative filtering



NaN and Empty String Counts for each column:
id                   0
title                0
is_paid              0
num_subscribers      0
avg_rating           0
avg_rating_recent    0
rating               0
num_reviews          0
is_wishlisted        0
tags                 0
dtype: int64

Total NaN and Empty String Count: 0


In [77]:
import pandas as pd
import numpy as np
import random

# Seed for reproducibility
np.random.seed(42)

# Number of users (1000)
num_users = 1000

# Sample learning styles, goals, feedback comments, and course difficulty levels
learning_styles = ["visual", "auditory", "kinesthetic"]
learning_goals = ["Improve skills", "Career advancement", "Learn new concepts", "Complete certification"]
feedback_comments = ["Great course", "Helpul but needs improvement", "Loved the interactive parts", "Could use more examples", "Too fast-paced", "Excellent content"]
difficulty_levels = ["Beginner", "Intermediate", "Advanced"]


# Assign random course titles from the previous dataset to users
course_titles = train_data['id'].tolist()

# Generate synthetic dataset for users
user_data = pd.DataFrame({
    'userId': [i % 10 for i in range(1, num_users+1)],
    'id': [random.choice(course_titles) for _ in range(num_users)],  # Assign random course titles to users
    'previous_test_scores': np.random.randint(50, 100, num_users),  # Test scores between 50 and 100
    'learning_style': [random.choice(learning_styles) for _ in range(num_users)],
    'learning_goals': [random.choice(learning_goals) for _ in range(num_users)],
    'course_duration': np.random.randint(10, 100, num_users),  # Total duration of each course
    'engagement_time_spent': np.random.randint(1, 100, num_users),  # Random time spent on the course
    'module_objectives': ["Understand basic concepts" for _ in range(num_users)],  # Can be dynamic
    'assessment_scores': np.random.randint(50, 100, num_users),  # Quiz/exam scores between 50 and 100
    'feedback_comments': [random.choice(feedback_comments) for _ in range(num_users)],
    'feedback_ratings': np.random.randint(1, 6, num_users),  # Ratings between 1 and 5
    'level_of_difficulty': [random.choice(difficulty_levels) for _ in range(num_users)]  # Course difficulty level
})

# Merge the user data with the previous courses dataset to assign the correct courseId based on course_title
user_data = pd.merge(user_data, train_data, on='id', how='left')

# Calculate percentage of the course completed based on engagement time
user_data['completion_percentage'] = (user_data['engagement_time_spent'] / user_data['course_duration'] * 100).round(2)

# Drop the engagement_time_spent and course_duration columns if needed
user_data.drop(columns=['course_duration', 'engagement_time_spent'], inplace=True)

user_data.to_csv('data.csv', index=False)

# Display the first 10 rows of the dataset
print(user_data.head(10))

   userId       id  previous_test_scores learning_style  \
0       1   950640                    88    kinesthetic   
1       2  2346552                    78         visual   
2       3  1341594                    64         visual   
3       4   838230                    92       auditory   
4       5  1390404                    57       auditory   
5       6   246502                    70         visual   
6       7  2059127                    88         visual   
7       8  2240390                    68    kinesthetic   
8       9  2413950                    72       auditory   
9       0  1818708                    60       auditory   

           learning_goals          module_objectives  assessment_scores  \
0          Improve skills  Understand basic concepts                 53   
1          Improve skills  Understand basic concepts                 99   
2          Improve skills  Understand basic concepts                 82   
3      Career advancement  Understand basic concep

In [60]:
user_data.shape

(1000, 20)

In [63]:
user_item_matrix = user_data.pivot_table(index='userId', columns='id', values='feedback_ratings', aggfunc='mean').fillna(0).astype(int)

In [64]:
user_similarity = cosine_similarity(user_item_matrix)

In [65]:
target_user_id = 4
target_user_index = user_item_matrix.index.get_loc(target_user_id)

In [70]:
user_similarities = user_similarity[target_user_index]
similar_user_indices = user_similarities.argsort()[::-1][1:]

In [72]:
reccomend_items = []

for user_index in similar_user_indices:
  rated_by_similar_user = user_item_matrix.iloc[user_index]
  not_rated_by_target_user = (rated_by_similar_user == 0) & (user_item_matrix.iloc[target_user_index] == 0)
  reccomend_items.extend(user_item_matrix.columns[not_rated_by_target_user][:10])

reccomend_items

[8467,
 8849,
 20185,
 23918,
 23972,
 25671,
 29432,
 30481,
 32908,
 34177,
 8467,
 8849,
 20185,
 23911,
 23918,
 23972,
 25671,
 29432,
 30481,
 33301,
 8467,
 8849,
 20185,
 23911,
 23972,
 25671,
 29432,
 30481,
 32908,
 33301,
 8467,
 8849,
 23911,
 23918,
 25671,
 29432,
 30481,
 32908,
 33301,
 38271,
 8467,
 8849,
 20185,
 23911,
 23918,
 23972,
 25671,
 30481,
 32908,
 33301,
 8467,
 8849,
 20185,
 23911,
 23918,
 23972,
 29432,
 30481,
 32908,
 33301,
 8467,
 8849,
 20185,
 23911,
 23918,
 23972,
 25671,
 29432,
 30481,
 32908,
 8849,
 23911,
 23918,
 23972,
 25671,
 29432,
 30481,
 32908,
 33301,
 34177,
 20185,
 23911,
 23918,
 23972,
 25671,
 29432,
 32908,
 33301,
 34177,
 42040]

In [74]:
recommended_items_details = user_data[user_data['id'].isin(reccomend_items)][['title', 'rating']]

In [75]:
recommended_items_details

,title,rating
48,New Business Essentials for Startups and Entre...,4.53956
52,Achieve excellence in restaurant service,4.66414
101,The Complete Introduction To Accounting and Fi...,4.40204
106,Buying Call and Put Options - Options beginner...,4.69244
144,The Lean Startup,3.78761
220,How to Write an Effective Research Paper,4.40924
254,Speak Up! Stand Out and be HAPPY,4.44269
256,The Lean Startup,3.78761
404,Get Your Book Into Print - Professional Format...,4.57155
407,Cold Calling: A New Approach,4.13555


##Hybrid Recommendation

In [ ]:
def hybrid_recommendations(train_data,target_user_id, item_anme,top_n=10):
  content_based_rec = content_based_recommendations(train_data,item_name,top_n)
  collaborative_rec = collaborative_filtering_recommendations(train_data,target_user_id,top_n)
  hybrid_rec = pd.concat([content_based_rec, collaborative_filtering_rec]).drop_duplicates()
  return hybrid_rec.head(10)